# Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (82%). There also exists, a smaller, but notable proportion of female players (17%). The rest are undisclosed/others (1%).

* Our peak age demographic falls between 20-24 (46%) with secondary groups falling between 15-19 (14%) and 25-29 (12%).

* The most popular and profitable item purchased is "Mourning Blade" at a price of $3.64.
-----

In [50]:
import os
import pandas as pd

os.chdir(os.path.join(os.environ["HOMEPATH"], 'Desktop', 'UCIRV201804DATA3-Class-Repository-DATA', '02-Homework', '04-Numpy-Pandas', 'Generators', 'HeroesOfPymoli', 'generated_data'))
print()
print('Make sure your input files (players_complete.csv, purchase_data_3.csv) are in this folder:')
print()
print(os.getcwd())


Make sure your input files (players_complete.csv, purchase_data_3.csv) are in this folder:

C:\Users\meifl\Desktop\UCIRV201804DATA3-Class-Repository-DATA\02-Homework\04-Numpy-Pandas\Generators\HeroesOfPymoli\generated_data


## Player Count

In [51]:
player = pd.read_csv('players_complete.csv')
# Read in player csv and find # of players
playerNum = player.shape[0]
pd.DataFrame({'Number of Players': [playerNum]})

,Number of Players
0,1163


## Purchasing Analysis (Total)

In [52]:
purchase = pd.read_csv('purchase_data_3.csv')
# Get number of unique items
unique_item = len(purchase['Item ID'].unique())
# Get average price
average_price = purchase.Price.sum() / purchase.shape[0]
# Get number of purchases
purchases = purchase.shape[0]
# Get total revenue
revenue = purchase.Price.sum()

pd.DataFrame({'Number of Unique Items': [unique_item],
             'Average Price': ['${:,.2f}'.format(average_price)],
             'Number of Purchases': [purchases],
             'Total Revenue': ['${:,.2f}'.format(revenue)]})

,Average Price,Number of Purchases,Number of Unique Items,Total Revenue
0,$2.92,78,64,$228.10


## Gender Demographics

In [53]:
# Groupby gender and get counts
total_count = purchase.groupby('Gender').count().iloc[:,0]
percentage = total_count / sum(total_count)
# Put in dataframe
df = pd.DataFrame({'Percentage of Players': percentage,
              'Total Count': total_count}).sort_values('Total Count', ascending = False)
df.index.name = ''
df

,Percentage of Players,Total Count
,,
Male,0.820513,64
Female,0.166667,13
Other / Non-Disclosed,0.012821,1



## Purchasing Analysis (Gender)

In [55]:
purchase_value = purchase.groupby('Gender').sum()['Price']
average_purchase = purchase_value / total_count

df2 = pd.DataFrame({'Purchase Count': total_count,
              'Average Purchase Price': average_purchase.map('${:,.2f}'.format),
              'Total Purchase Value': purchase_value.map('${:,.2f}'.format)})
              #'Normalized Totals': [0, 0, 0]})
df2 = df2[['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']] 
           #'Normalized Totals']]
df2

,Purchase Count,Average Purchase Price,Total Purchase Value
Gender,,,
Female,13,$3.18,$41.38
Male,64,$2.88,$184.60
Other / Non-Disclosed,1,$2.12,$2.12


## Age Demographics

In [56]:
# Get age bin dataframe
bins = [0, 10, 14, 19, 24, 29, 34, 39, 40]
labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '+40']
df3 = pd.cut(purchase['Age'], bins=bins, labels=labels).to_frame().Age.value_counts(sort = False).to_frame()
df3['Percentage'] = round(df3.Age / df3.Age.sum(), 2)
df3.columns = ['Total Count', 'Percentage of Players']
df3 = df3[['Percentage of Players', 'Total Count']]
df3

,Percentage of Players,Total Count
<10,0.06,5
10-14,0.04,3
15-19,0.14,11
20-24,0.46,36
25-29,0.12,9
30-34,0.09,7
35-39,0.08,6
+40,0.01,1


## Purchasing Analysis (Age)

In [57]:
df4 = pd.cut(purchase['Age'], bins=bins, labels=labels).to_frame().Age.value_counts(sort = False).to_frame()
df4.columns = ['Purchase Count']
df4['Total Purchase Value'] = purchase.groupby(pd.cut(purchase['Age'], bins=bins, labels=labels)).sum()['Price']
df4['Average Purchase Price'] = (df4['Total Purchase Value'] / df4['Purchase Count']).map('${:,.2f}'.format)
df4['Total Purchase Value'] = df4['Total Purchase Value'].map('${:,.2f}'.format)
#df4['Normalized Totals'] = 0
df4 = df4.iloc[:, [0,2,1]]
df4

,Purchase Count,Average Purchase Price,Total Purchase Value
<10,5,$2.76,$13.82
10-14,3,$2.99,$8.96
15-19,11,$2.76,$30.41
20-24,36,$3.02,$108.89
25-29,9,$2.90,$26.11
30-34,7,$1.98,$13.89
35-39,6,$3.56,$21.37
+40,1,$4.65,$4.65


## Top Spenders

In [58]:
df5 = purchase.groupby('SN')['Price'].sum().sort_values(ascending = False).to_frame()
df5.columns = ['Total Purchase Value']
df5['Purchase Count'] = purchase.groupby('SN')['SN'].count()
df5['Average Purchase Price'] = (df5['Total Purchase Value'] / df5['Purchase Count']).map('${:,.2f}'.format)
df5['Total Purchase Value'] = df5['Total Purchase Value'].map('${:,.2f}'.format)
df5 = df5.iloc[:, [1,2,0]]
df5.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Sundaky74,2,$3.71,$7.41
Aidaira26,2,$2.56,$5.13
Eusty71,1,$4.81,$4.81
Chanirra64,1,$4.78,$4.78
Alarap40,1,$4.71,$4.71


## Most Popular Items

In [59]:
df6 = purchase.groupby(['Item ID', 'Item Name'])['Item ID'].count().sort_values(ascending = False).to_frame()
df6.columns = ['Purchase Count']
df6['Total Purchase Value'] = purchase.groupby(['Item ID', 'Item Name'])['Price'].sum()
df6['Item Price'] = (df6['Total Purchase Value'] / df6['Purchase Count']).map('${:,.2f}'.format)
df6['Total Purchase Value'] = df6['Total Purchase Value'].map('${:,.2f}'.format)
df6 = df6.iloc[:, [0,2,1]]
df6.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
94,Mourning Blade,3,$3.64,$10.92
98,"Deadline, Voice Of Subtlety",2,$1.29,$2.58
117,"Heartstriker, Legacy of the Light",2,$4.71,$9.42
111,Misery's End,2,$1.79,$3.58
154,Feral Katana,2,$4.11,$8.22


## Most Profitable Items

In [60]:
df7 = purchase.groupby(['Item ID', 'Item Name'])['Price'].sum().sort_values(ascending = False).to_frame()
df7.columns = ['Total Purchase Value']
df7['Purchase Count'] = purchase.groupby(['Item ID', 'Item Name'])['Item ID'].count()
df7['Item Price'] = (df7['Total Purchase Value'] / df7['Purchase Count']).map('${:,.2f}'.format)
df7['Total Purchase Value'] = df7['Total Purchase Value'].map('${:,.2f}'.format)
df7 = df7.iloc[:, [1,2,0]]
df7.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
94,Mourning Blade,3,$3.64,$10.92
117,"Heartstriker, Legacy of the Light",2,$4.71,$9.42
93,Apocalyptic Battlescythe,2,$4.49,$8.98
90,Betrayer,2,$4.12,$8.24
154,Feral Katana,2,$4.11,$8.22
